In [4]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import os
import collections

import nltk
import numpy as np
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import codecs

np.random.seed(42)

Using TensorFlow backend.


In [0]:
!mkdir data
!wget https://raw.githubusercontent.com/chen0040/keras-sentiment-analysis-web-api/master/demo/data/umich-sentiment-train.txt -P ./data

mkdir: cannot create directory ‘data’: File exists
--2018-11-05 05:31:38--  https://raw.githubusercontent.com/chen0040/keras-sentiment-analysis-web-api/master/demo/data/umich-sentiment-train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447539 (437K) [text/plain]
Saving to: ‘./data/umich-sentiment-train.txt’

umich-sentiment-tra 100%[===================>] 437.05K  --.-KB/s    in 0.08s   

2018-11-05 05:31:38 (5.14 MB/s) - ‘./data/umich-sentiment-train.txt’ saved [447539/447539]



In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip -P ./data
!unzip ./data/glove.6B.zip -d ./data

--2018-11-05 05:53:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-11-05 05:53:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘./data/glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  6.84MB/s    in 74s     

2018-11-05 05:54:19 (11.1 MB/s) - ‘./data/glove.6B.zip’ saved [862182613/862182613]

Archive:  ./data/glove.6B.zip
  inflating: ./data/glove.6B.50d.txt  
  inflating: ./data/glove.6B.100d.txt  
  inflating: ./data/glove.6B.200d.txt  
  inflating: ./data/glove.6B.300d.txt  


In [0]:
INPUT_FILE = "./data/umich-sentiment-train.txt"
GLOVE_MODEL = "./data/glove.6B.300d.txt"
LOG_DIR = "./logs"
VOCAB_SIZE = 5000
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 10

In [5]:
counter = collections.Counter()
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    maxlen = 0
    for line in fin:
        _, sent = line.strip().split("\t")
        try:
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        except LookupError:
            print("Englisth tokenize does not downloaded. So download it.")
            nltk.download("punkt")
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            counter[word] += 1

word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}

xs, ys = [], []
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    for line in fin:
        label, sent = line.strip().split("\t")
        ys.append(int(label))
        words = [x.lower() for x in nltk.word_tokenize(sent)]
        wids = [word2index[word] for word in words]
        xs.append(wids)

X = pad_sequences(xs, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3,
                                                random_state=42)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

(4960, 42) (2126, 42) (4960, 2) (2126, 2)


In [6]:
# load GloVe vectors
word2emb = {}
with codecs.open(GLOVE_MODEL, "r", encoding="utf-8") as fglove:
    for line in fglove:
        cols = line.strip().split()
        word = cols[0]
        embedding = np.array(cols[1:], dtype="float32")
        word2emb[word] = embedding

embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
    try:
        embedding_weights[index, :] = word2emb[word]
    except KeyError:
        pass

model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen,
                    weights=[embedding_weights],
                    trainable=True))
model.add(Dropout(0.2))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS,
                 activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 42, 300)           698700    
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 300)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40, 256)           230656    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 929,870
Trainable params: 929,870
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=[TensorBoard(LOG_DIR)],
                    validation_data=(Xtest, Ytest))              

# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

Train on 4960 samples, validate on 2126 samples
Epoch 1/10
4960/4960 [==============================] - 3s 595us/step - loss: 0.1239 - acc: 0.9470 - val_loss: 0.0325 - val_acc: 0.9873
Epoch 2/10
4960/4960 [==============================] - 1s 197us/step - loss: 0.0166 - acc: 0.9964 - val_loss: 0.0238 - val_acc: 0.9929
Epoch 3/10
4960/4960 [==============================] - 1s 193us/step - loss: 0.0072 - acc: 0.9986 - val_loss: 0.0219 - val_acc: 0.9920
Epoch 4/10
4960/4960 [==============================] - 1s 194us/step - loss: 0.0046 - acc: 0.9992 - val_loss: 0.0190 - val_acc: 0.9934
Epoch 5/10
4960/4960 [==============================] - 1s 196us/step - loss: 0.0031 - acc: 0.9992 - val_loss: 0.0190 - val_acc: 0.9929
Epoch 6/10
4960/4960 [==============================] - 1s 196us/step - loss: 0.0018 - acc: 0.9998 - val_loss: 0.0209 - val_acc: 0.9920
Epoch 7/10
4960/4960 [==============================] - 1s 197us/step - loss: 0.0020 - acc: 0.9998 - val_loss: 0.0188 - val_acc: 0.9934
